In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pyeCAP
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import traceback
import os
import dask.array as da

In [ ]:
"Import meta data"
"""Get HF meta data from excel file"""
excel_path = r'E:\HF_Block\20240904_HFBlock_Pig05\20240904_HFBlock_05_Parameters_modified.xlsx'
metaDF = pd.read_excel(excel_path, sheet_name = 'HF05_metaDF')

"""Remove rows that are not needed for current analysis and reset index of metaDF"""
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'HF Only'].index, inplace = True)  #Removed HF only tanks as they did not have any stim parameters
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Control'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'X'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Thresholding'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Contact ID'].index, inplace = True)
metaDF.reset_index(inplace = True)

"TDT Data Import - List"
tdt_path = r'E:\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Lab Path
#tdt_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Local Path 
tdt_file_list_all = os.listdir(tdt_path)
tdt_file_list_all.remove('desktop.ini')

ADI_path = r'E:\HF_Block\20240904_HFBlock_Pig05' #Lab Path
#ADI_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05' #Local path
ADI_file = r'\20240904_HFBlock_Pig05_stats_modified.mat'

raw_phys = pyeCAP.Phys(ADI_path + ADI_file)
ws_stripped_names = [i.strip() for i in raw_phys.ch_names] #This line just removes the white spaces around the names imported
raw_phys = raw_phys.set_ch_names(ws_stripped_names)

"""Ephys data instantiation"""
tdt_path_list = [tdt_path + '\\' + fileNAME for fileNAME in metaDF['TDT Tank']]

#Data Streams
raw_ECAP = pyeCAP.Ephys(tdt_path_list, stores = 'ECAP')
raw_ECAP = raw_ECAP.remove_ch(['ECAP 6', 'ECAP 7', 'ECAP 8'])

"""Changes the stim multi-index to utilize the Tank names from tdt_file_list instead [0,1,2,3...]"""
stim = pyeCAP.Stim(tdt_path_list)
stimDF = stim.parameters
#tank_to_params_dict = { tank:params for (tank,params) in zip(metaDF['TDT Tank'].to_list(), stim.parameters.groupby(level=0).groups.values() ) }
#params_to_tank_dict = { param:tank for (param,tank) in zip(stim.parameters.groupby(level=0).groups.keys(), metaDF['TDT Tank'].to_list()) }

"""ECAP/EMG/TriCAP object instantiation"""
ECAPr = pyeCAP.ECAP(raw_ECAP, stim)
PHYS = pyeCAP.ECAP(raw_phys,stim)
raw_ECG_array = np.squeeze( raw_phys.array[raw_phys._ch_to_index('EKG Raw')].compute() )

In [ ]:
"""Band-pass Filter"""
ECG_fs = 1000
nyq_rate = ECG_fs / 2

cutoffs = [5,15]
b,a = signal.butter(4, cutoffs, btype='bandpass', fs = ECG_fs)

"""Code which replaces any NaN values by simply replacing them with the first value after the NaN's occur"""
nan_idx = np.argwhere(np.isnan(raw_ECG))
np.nan_to_num(raw_ECG, copy=False, nan=raw_ECG[nan_idx[-1] + 1])

ECG_BP_filt = signal.filtfilt(b, a, raw_ECG)
ECG_dvt_filt = np.diff(ECG_BP_filt,append=1)
ECG_sq = np.square(ECG_dvt_filt)
ECG_moving_average = uniform_filter1d(ECG_sq,size=150)

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))
ax.plot(filt_data[0].compute())
plt.show()

In [ ]:
raw_phys.sample_rate

In [ ]:
raw_phys.time()[0:100].compute()

In [ ]:
data = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

In [ ]:
new_array = [da.squeeze(da.stack([d1, d2, d3])) for d1,d2,d3 in zip(filt_data,data_diff_sq,data_moving_average)]#, data_diff_sq, data_moving_average)

In [ ]:
new_array

In [ ]:
raw_phys.data

In [ ]:
combined_array = [da.concatenate([d1,d2], axis=0) for d1,d2 in zip(raw_phys.data,new_array)]

In [ ]:
combined_array